In [ ]:
import pygame as pg
import random
import sys
import traceback
import time

# 颜色定义
#国家
YELLOW = (255, 255, 0)  # 黄色
SILVER = (192, 192, 192)  # 银色
GRAY = (128, 128, 128)  # 灰色
DEEPSKYBLUE = (0, 191, 255)  # 深天蓝色
LIGHTGREEN = (144, 238, 144)  # 浅绿色
#特殊显示
GOLD = (255, 215, 0)  #鼠标点击
BLACK = (0, 0, 0) #线和数字
CRIMSON = (220, 20, 60)  # 深红色

LIGHTYELLOW = (255, 255, 224)
SKY_BLUE = (135, 206, 235)
NAVYBLUE = (0, 0, 128)
GREEN = (0, 255, 0)

DARKORANGE = (255, 140, 0)  # 暗橙色
MEDIUMPURPLE = (147, 112, 219)  # 中紫色






# 六边形大小常数
HEX_Y_SIZE = 21
HEX_X_SIZE = 18
MAP_WIDTH = 800
MAP_HEIGHT = 600
height = int( MAP_HEIGHT // (0.75 * HEX_Y_SIZE) )
width = int( MAP_WIDTH // HEX_X_SIZE )

SCREEN_WIDTH = 800
SCREEN_HEIGHT = 660

min_tiles = 20
landlist = [] #

#创建对应关系 tile number ： countland
tilelanddict = {}
for y in range(height):
        for x in range(width):
            tilelanddict[(x,y)] = 0

# 获取六边形地图位置的函数
def getHexMapPos(x, y):
    X_LEN = HEX_X_SIZE // 2
    Y_LEN = HEX_Y_SIZE // 2
    if y % 2 == 0:
        base_x = X_LEN * 2 * x 
        base_y = Y_LEN * 3 * (y // 2)
    else:
        base_x = X_LEN * 2 * x + X_LEN
        base_y = Y_LEN * 3 * (y // 2) + Y_LEN // 2 + Y_LEN
    return (base_x, base_y)

#定义Vector2d类，为isInTriangle做铺垫
class Vector2d:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def minus(self, other):
        return Vector2d(self.x - other.x, self.y - other.y)
    
    def crossProduct(self, other):
        return self.x * other.y - self.y * other.x


# isInTriangle判断点在三角形内还是形外，为getHexMapIndex方法做铺垫
def isInTriangle(x1, y1, x2, y2, x3, y3, x, y):
    A = Vector2d(x1, y1)
    B = Vector2d(x2, y2)
    C = Vector2d(x3, y3)
    P = Vector2d(x, y)
    PA = A.minus(P)
    PB = B.minus(P)
    PC = C.minus(P)
    t1 = PA.crossProduct(PB)
    t2 = PB.crossProduct(PC)
    t3 = PC.crossProduct(PA)
    if (t1 * t2 >= 0) and (t1 * t3 >= 0):
        return True
    return False

#getHexMapIndex将鼠标坐标对应到地图六边形坐标
def getHexMapIndex(x, y):
    X_LEN = HEX_X_SIZE // 2
    Y_LEN = HEX_Y_SIZE // 2
    tmp_x, offset_x = divmod(x, HEX_X_SIZE)
    tmp_y, offset_y = divmod(y, Y_LEN * 3)
    map_x, map_y = 0, 0
    if offset_y <= (Y_LEN + Y_LEN//2):
        if offset_y >= Y_LEN//2:
            map_x, map_y = tmp_x, tmp_y * 2
        else:
            triangle_list = [(0, 0, 0, Y_LEN//2, X_LEN, 0),
                             (0, Y_LEN//2, X_LEN, 0, HEX_X_SIZE, Y_LEN//2),
                             (X_LEN, 0, HEX_X_SIZE, 0, HEX_X_SIZE, Y_LEN//2)]
            map_list = [(tmp_x - 1, tmp_y * 2 -1), (tmp_x, tmp_y * 2), (tmp_x, tmp_y * 2 -1)]
            for i, data in enumerate(triangle_list):
                if isInTriangle(*data, offset_x, offset_y):
                    map_x, map_y = map_list[i]
                    break
    elif offset_y >= HEX_Y_SIZE:
        if offset_x <= X_LEN:
            map_x, map_y = tmp_x - 1, tmp_y * 2 + 1
        else:
            map_x, map_y = tmp_x, tmp_y *2 + 1
    else:
        triangle_list = [(0, Y_LEN + Y_LEN//2, 0, HEX_Y_SIZE, X_LEN, HEX_Y_SIZE),
                         (0, Y_LEN + Y_LEN//2, X_LEN, HEX_Y_SIZE, HEX_X_SIZE, Y_LEN + Y_LEN//2),
                         (X_LEN, HEX_Y_SIZE, HEX_X_SIZE, Y_LEN + Y_LEN//2, HEX_X_SIZE, HEX_Y_SIZE)]
        map_list = [(tmp_x - 1, tmp_y * 2 + 1), (tmp_x, tmp_y * 2), (tmp_x, tmp_y *2 + 1)]
        for i, data in enumerate(triangle_list):
            if isInTriangle(*data, offset_x, offset_y):
                map_x, map_y = map_list[i]
                break
    if map_x == 0 and map_y == 0:
        print('pos[%d, %d](%d, %d) base[%d, %d] off[%d, %d] ' % (map_x, map_y, x, y, tmp_x, tmp_y, offset_x, offset_y))
    return (map_x, map_y)

def click_land(click_x, click_y):
    tile_x, tile_y = getHexMapIndex(click_x, click_y)
    if (tile_x, tile_y) in tilelanddict.keys():
        land_id = tilelanddict[(tile_x, tile_y)]
        return land_id
    else:
        return 0


# 绘制背景六边形的函数
def drawBackgroundHex(surface, width, height, bg_map):
    Y_LEN = HEX_Y_SIZE // 2
    X_LEN = HEX_X_SIZE // 2

    # 填充背景色
    surface.fill(LIGHTYELLOW)

    # 绘制所有的六边形
    for y in range(height):
        for x in range(width):
            color = LIGHTYELLOW  # 默认颜色
            if bg_map[y][x] == 'EMPTY':
                color = LIGHTYELLOW
            elif bg_map[y][x] == 'ACTIVE':
                color = SKY_BLUE
            elif bg_map[y][x] == 'ADD_DICE':
                color = BLACK
            elif bg_map[y][x] == 'RANGE':
                color = NAVYBLUE
            elif bg_map[y][x] == 'SELECT':
                color = GREEN
            elif bg_map[y][x] == 'ATTACK':
                color = GOLD
            elif bg_map[y][x] == 1:
                color = YELLOW
            elif bg_map[y][x] == 2:
                color = SILVER
            elif bg_map[y][x] == 3:
                color = GRAY
            elif bg_map[y][x] == 4:
                color = DEEPSKYBLUE
            elif bg_map[y][x] == 5:
                color = LIGHTGREEN
            
            base_x, base_y = getHexMapPos(x, y)
            points = [(base_x, base_y + Y_LEN // 2 + Y_LEN), (base_x, base_y + Y_LEN // 2),
                      (base_x + X_LEN, base_y), (base_x + X_LEN * 2, base_y + Y_LEN // 2),
                      (base_x + X_LEN * 2, base_y + Y_LEN // 2 + Y_LEN), (base_x + X_LEN, base_y + Y_LEN * 2)]
            pg.draw.polygon(surface, color, points)
   
    # 绘制六边形的黑色边框
    for y in range(height):
        for x in range(width):
            if y % 2 == 1 and x == width - 1:
                continue
            
            base_x, base_y = getHexMapPos(x, y)
            points = [
                (base_x, base_y + Y_LEN // 2 + Y_LEN),
                (base_x, base_y + Y_LEN // 2),
                (base_x + X_LEN, base_y),
                (base_x + X_LEN * 2, base_y + Y_LEN // 2),
                (base_x + X_LEN * 2, base_y + Y_LEN // 2 + Y_LEN),
                (base_x + X_LEN, base_y + Y_LEN * 2),
                (base_x, base_y + Y_LEN // 2 + Y_LEN)  # 再次添加第一个点，以便绘制最后一条边
            ]
            
            if y % 2 == 0:  # 宽的一行
                neigh_line_dict = {(x-1, y):(points[0], points[1]), (x-1, y-1):(points[1], points[2]), (x, y-1):(points[2], points[3]), 
                                   (x+1, y):(points[3], points[4]), (x, y+1):(points[4], points[5]), (x-1, y+1):(points[5], points[6])}
            else:  # 窄的一行
                neigh_line_dict = {(x-1, y):(points[0], points[1]), (x, y-1):(points[1], points[2]), (x+1, y-1):(points[2], points[3]), 
                                    (x+1, y):(points[3], points[4]), (x+1, y+1):(points[4], points[5]), (x, y+1):(points[5], points[6])}
            
            tile_num = tilelanddict[(x,y)]
            if tile_num != 0: #已经生成的地块
                for key in neigh_line_dict.keys():
                    if key not in tilelanddict.keys() or tilelanddict[key] != tile_num:                     #邻居和自己不是同一块地
                        start_point = neigh_line_dict[key][0]
                        end_point = neigh_line_dict[key][1]
                        pg.draw.line(surface, BLACK, start_point, end_point)
    
    #绘制land上的骰子数
    for landid in landsdict.keys():
        dicenum = landsdict[landid].dicenum #要画上去的数字
        painttile_x, painttile_y = landsdict[landid].dicepos
        paintbase_x, paintbase_y = getHexMapPos(painttile_x, painttile_y)
        paintbase_x = paintbase_x + X_LEN
        paintbase_y = paintbase_y + Y_LEN #数字的坐标中心
        # 使用字体对象渲染数字
        text_surface = my_font.render(str(dicenum), True, BLACK)  # 第二个参数True表示开启抗锯齿
        # 计算文本Surface的新位置，使其在六边形的中心
        text_rect = text_surface.get_rect(center=(paintbase_x, paintbase_y))
        # 将文本Surface绘制到主Surface上
        surface.blit(text_surface, text_rect)

    try:
        showboard.show()
    except:
        pass

def land_countrynum_map(countrynum, landpercoun):
    countryrandomlist = []
    countryrandomlist = list(range(1, countrynum + 1)) * landpercoun
    random.shuffle(countryrandomlist)
    return countryrandomlist

def land_dice_map(averdice, countrynum, landpercoun, countryrandomlist):
    dicerandomlist = [0] * (countrynum * landpercoun)
    for country in range(1, countrynum+1):
        dice = dice_percoun(country, averdice, landpercoun)
        indices_of_country = [index for index, value in enumerate(countryrandomlist) if value == country] #把国家列表中国家对应的土地序号找到
        for coun_index in indices_of_country:
            dicerandomlist[coun_index] = dice[indices_of_country.index(coun_index)]
    
    return dicerandomlist

def dice_percoun(countrynum, averdice, landpercoun, mindice=1, maxdice=8 ):
    totaldice = averdice * landpercoun + countrynum * 2
    #随机生成
    dicelist = []
    for i in range(0, landpercoun):
        dicelist.append(random.randint(mindice, maxdice))
    length = sum(dicelist) - totaldice
    
    while length != 0:
        if length > 0:
            index = random.randint(0, landpercoun-1)
            if dicelist[index] > 1:
                dicelist[index] -= 1 
                length -= 1
        elif length < 0:
            index = random.randint(0, landpercoun-1)
            if dicelist[index] < 8:    
                dicelist[index] += 1  
                length += 1

    return dicelist


    

pg.font.init()  # 初始化字体系统
my_font = pg.font.SysFont('Arial', 16)  # 创建一个字体对象，这里使用了Arial字体，大小为24

# 测试背景地图数据，你可以根据自己的需要修改这个数据
bg_map = [['EMPTY' for _ in range(width)] for _ in range(height)]  # 示例数据

# 游戏主循环
countland = 0
countrynum = 4
landpercoun = 20
landnum = countrynum * landpercoun
clicklandnum = 0
averdice = 4

#初始参数生成
countryrandomlist = land_countrynum_map(countrynum, landpercoun) #land所属国家
dicerandomlist = land_dice_map(averdice, countrynum, landpercoun, countryrandomlist) #每个land上的骰子数
landsdict = {}

#写个map generation函数，把所有的统合起来
try:

    # 初始化 Pygame
    pg.init()
    screen = pg.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
    
    #生成地图
    clicklandnum += landnum
    starting_x, starting_y = random.randint(0, width - 1), random.randint(0, height - 1)
    AllLandGenerator(starting_x, starting_y, clicklandnum)
    landadjdict = AllLandAdj(landnum)
    print(landadjdict)
    
    #补充每个土地实例，需要有 包含的tile(ok) 骰子(ok) country(ok) 接壤land countland(ok)
    for key in landsdict.keys():
        landsdict[key].update_adjland(landadjdict[key])
    
    #生成计数工具 和· 国家对象
    tools = Tools(1)
    countrydict = {}
    for countryid in range(1, countrynum + 1):
        countrydict[countryid] = Country(countryid, landsdict)
    
    #生成显示工具
    showboard = ShowBoard(countrynum)
    drawBackgroundHex(screen, width, height, bg_map)
    pg.display.flip()
    #showboard.show()
    
    running = True
    while running:   
        for event in pg.event.get():
            if event.type == pg.QUIT:
                running = False
            elif event.type == pg.MOUSEBUTTONDOWN:
                if event.button == 1:
                    click_x, click_y = event.pos
                    afterclick(click_x, click_y)                 
                             
                    drawBackgroundHex(screen, width, height, bg_map)
                    pg.display.flip()             
                
except Exception as e:
        # 打印异常的完整堆栈跟踪
        traceback.print_exc()
        print(f"An error occurred: {e}", file=sys.stderr)
        #running = False
    
    

# 退出 Pygame
pg.quit()


{1: {2, 3, 76}, 2: {1, 3, 5}, 3: {1, 2, 4, 5, 75, 76}, 4: {3, 5, 6, 75, 47, 48}, 5: {2, 3, 4, 6}, 6: {4, 5, 7}, 7: {6, 8, 9, 48}, 8: {7, 9, 53, 54}, 9: {7, 8, 10, 48}, 10: {9, 11, 12, 48, 53}, 11: {10, 12, 15, 51, 53}, 12: {10, 11, 13, 14, 15, 48}, 13: {40, 41, 12, 14}, 14: {38, 39, 12, 13, 15, 16}, 15: {11, 12, 14, 16, 51}, 16: {14, 15, 17, 50, 51}, 17: {16, 18, 50, 20, 21}, 18: {17, 19, 20, 56, 57}, 19: {18, 20, 57}, 20: {17, 18, 19, 21}, 21: {17, 20, 22}, 22: {49, 50, 21, 23}, 23: {49, 22, 24}, 24: {49, 23, 25, 26}, 25: {34, 24, 26, 27}, 26: {58, 24, 25, 27}, 27: {33, 26, 25, 58, 28}, 28: {33, 58, 27, 29, 30}, 29: {58, 28, 30}, 30: {32, 33, 28, 29, 31}, 31: {32, 67, 30}, 32: {33, 67, 35, 30, 31}, 33: {32, 34, 35, 27, 28, 30}, 34: {33, 35, 37, 25}, 35: {32, 33, 34, 67, 36, 37}, 36: {66, 35, 37, 67, 59}, 37: {34, 35, 36, 38, 39, 59}, 38: {37, 39, 14, 49, 50}, 39: {37, 38, 40, 14, 59}, 40: {39, 41, 13, 59, 60}, 41: {40, 42, 13}, 42: {41, 43, 45, 60}, 43: {68, 69, 42, 44, 45, 60}, 44: {

In [2]:
#landsdict[land_id].click_color()

def afterclick(click_x, click_y):
    if tools.countturn % countrynum != 0:
        countrymove = tools.countturn % countrynum
    elif tools.countturn % countrynum == 0:
        countrymove = countrynum
    
    if tools.click_allow == 0:
        pass

    elif tools.click_allow == 1:
        
        if click_y > MAP_HEIGHT:                #结束回合
            # enter next term 加骰子 term+1
            #还缺一个随机加骰子的算法
            for land in landsdict.values():   #避免在selected情况下结束回合地块颜色没法跳回去
                land.update_color()
                
            countrydict[countrymove].add_dice()
            tools.countturn += 1
            
        elif click_y <= MAP_HEIGHT:
            
            land_id = click_land(click_x, click_y)
            if land_id == 0:
                pass
            elif land_id > 0:
                if  tools.landstate == 0: # attacker
                    if landsdict[land_id].country != countrymove:
                        pass
                    else:
                        tools.sword = land_id
                        landsdict[land_id].update_color("SELECT")
                        tools.landstate = 1
                elif tools.landstate == 1: # being attacked
                    #todo
                    if landsdict[land_id].country == countrymove: #点击另一片自己的land
                        landsdict[tools.sword].update_color()
                        tools.sword = land_id
                        landsdict[land_id].update_color("SELECT")
                    elif land_id not in landadjdict[tools.sword] and landsdict[land_id].country != countrymove: #不接壤，归位
                        landsdict[tools.sword].update_color()
                        tools.landstate = 0
                        tools.sword = 0          
                    elif land_id in landadjdict[tools.sword] and landsdict[land_id].country != countrymove:#接壤，攻击
                        comparsion(landsdict[tools.sword], landsdict[land_id] )
                        tools.landstate = 0

def comparsion(attacker, defender):
    attacker_dice = attacker.throwdice()
    defender_dice = defender.throwdice()
    
    if attacker_dice > defender_dice:
        defender.defend_lose(attacker)
        attacker.attack_win()
    elif attacker_dice <= defender_dice:
        attacker.attack_lose()
        defender.defend_win()


#定义Map相关的一些参数
class Tools:
    def __init__(self, something):
        self.click_allow = 1 # flag用于区分阶段，0点击无效，1点击有效
        self.landstate = 0 # flag用于区分land选中的状态，攻击还是被攻击
        self.countturn = 1 #每次结束回合加1，用来判定回合国家
        self.sword =  0 #attacker
        self.land_country_dict = {x:0 for x in range(1, landnum+1 )}
        self.state_color_dict= {1:YELLOW, 2:SILVER, 3:GRAY, 4:DEEPSKYBLUE, 5:LIGHTGREEN, 'SELECT':GREEN, "ADD_DICE":BLACK}

    def landcountry_update(self, landsdict): #构建land和country的对应关系，便于计算land和连通land
        for land in landsdict:
            self.land_country_dict[land.id] = land.country

#定义country类，在加骰子和添加扩展性方面或许有用。依托于landsdict，本质上是将复杂的计算打包进了country里面
class Country:
    def __init__(self, countryid, landsdict):
        self.id = countryid
        self.landset = set() #国家中包含哪些land
        for land in landsdict.values():
            if land.country == self.id:
                self.landset.add(land.id)
        self.land_num = 0
        self.adjland_num = 0
    
    def land_earn(self, land):
        self.landset.add(land.id)

    def land_lose(self, land):
        self.landset.remove(land.id)

    def land_print(self):
        print(self.landset)

    def land_count(self):     #计算自己有多少土地
        return len(self.landset)

    def adjland_count(self):    #计算自己有多少连通土地，这里有用到landsdict
        self.landlist = list(self.landset)
        #print(self.landlist)
        self.landislandlist = []
        
        def waiting_to_final(landid):
            #print(landid)
            self.landlist.remove(landid)
            self.neigh_lands_id = list(landsdict[landid].adjland)
            #print(landid,"has neighbour", self.neigh_lands_id)
            for self.neigh_land_id in self.neigh_lands_id:
                if landsdict[self.neigh_land_id].country == self.id and landsdict[self.neigh_land_id].id not in self.waitinglist:
                    #print(landid,"OK neighbour", landsdict[self.neigh_land_id].id)
                    self.waitinglist.append(self.neigh_land_id)
            self.finallist.append(landid)
        
        while len(self.landlist) > 0:
            self.waitinglist = []
            self.finallist = []
            landid_chosen = random.choice(self.landlist)
            self.waitinglist.append(landid_chosen)
            while len(self.finallist) !=  len(self.waitinglist):
                self.tochoose_list = list(set(self.waitinglist) - set(self.finallist))
                #print("tochoose_list", self.tochoose_list)
                waiting_chosen = random.choice(self.tochoose_list)
                waiting_to_final(waiting_chosen)
            
            #print("finallist", self.finallist)
            self.landislandlist.append(self.finallist)
            #print("self.landislandlist", self.landislandlist)

        islandnumlist = []
        for self.islandlist in self.landislandlist:
            islandnumlist.append(len(self.islandlist))
        
        if len(islandnumlist) == 0:
            return 0
        else:
            return max(islandnumlist)
           
    def add_dice(self):     #执行随机加骰子操作，这里有用到landsdict
        
        self.addnum = self.adjland_count()
        print(self.addnum)
        while self.addnum > 0:
            randomlandid = random.choice(list(self.landset))
            if landsdict[randomlandid].dicenum < landsdict[randomlandid].dicemax:
                landsdict[randomlandid].add_dice()
                self.addnum -= 1
                

#定义land类
class Land:
    def __init__(self, countland, intiles, countrynum, dicenum ):
        self.id = countland
        self.intiles = list(intiles) #内部的tile坐标
        self.adjland = set()
        self.dicenum = dicenum
        self.country = countrynum
        self.dicepos = random.choice(self.intiles)
        self.adjlandbro = set()
        self.dicemax = 8
        

    def update_adjland(self, adjland):
        self.adjland = adjland

    def update_adjlandbro(self):
        self.adjlandbro.clear()
        for landid in self.adjland:
            if landsdict[landid].country == self.country:
                self.adjlandbro.add(landid)
                
    def update_color(self, color_type=None):
        if color_type is None:
            color_type = self.country
        for x, y in self.intiles:
            bg_map[y][x] = color_type
        
    def attack_win(self):
        self.dicenum = 1
        self.update_adjlandbro()
        self.update_color()
        

    def attack_lose(self):
        self.dicenum = 1
        self.update_color()

    def defend_win(self):
        self.update_color()
        pass

    def defend_lose(self, land):
        countrydict[self.country].land_lose(self)
        self.country = land.country
        countrydict[self.country].land_earn(self)
        self.dicenum = land.dicenum - 1
        self.update_adjlandbro()
        self.update_color()
        
        #countrydict[self.country].land_print()

    def throwdice(self):
        if self.dicenum == 1:
            return 0
        else:
            self.sum = 0
            for i in range(0, self.dicenum):
                dicenum = random.randint(1, 6)
                self.sum += dicenum
            return self.sum

    def add_dice(self):     #这段主要是加骰子的效果显示
        
        self.update_color("ADD_DICE")
        self.dicenum += 1
        drawBackgroundHex(screen, width, height, bg_map)
        pg.display.flip()
        pg.time.delay(100)
        self.update_color()
        drawBackgroundHex(screen, width, height, bg_map)
        pg.display.flip()
        
class ShowBoard:
    def __init__(self, countrynum):
        self.countrynum = countrynum
        self.rect_width = 80
        self.rect_height = 20
        self.color_size = 20
        self.landdict = {}
        self.adjlanddict = {}
        self.countrymove = 1

    def country_update(self):
        for countryid in range(1, countrynum+1):
            self.landdict[countryid] = countrydict[countryid].land_count()
            self.adjlanddict[countryid] = countrydict[countryid].adjland_count()

    def country_move(self):
        if tools.countturn % countrynum != 0:
            self.countrymove = tools.countturn % countrynum
        elif tools.countturn % countrynum == 0:
            self.countrymove = countrynum
        #print(self.countrymove)
    #在像素0 - MAP_WIDTH -40 || MAP_HEIGHT - SCREEN_HEIGHT 的空间内定义 self.countrynum
    #个小rect，rect长宽分别是self.rect_width，self.rect_width，由色块 + 数字构成
    #色块颜色是color.id，数字为 self.adjlanddict[id]  / self.landdict[id]
    def show(self):
        self.country_update()
        self.show_y = MAP_HEIGHT + 0.5 * (SCREEN_HEIGHT - MAP_HEIGHT )
        self.show_x = 0.5 * (MAP_WIDTH - 40 - countrynum * self.rect_width)
        self.show_x_list = []
        for countryid in range(1, self.countrynum + 1): #改变国家数据显示
            pg.draw.rect(screen, tools.state_color_dict[countryid], (self.show_x, self.show_y, self.color_size, self.color_size))
            self.textshow = str(self.adjlanddict[countryid]) + " / " + str(self.landdict[countryid])
            text = my_font.render(self.textshow, True, (0, 0, 0))
            screen.blit(text, (self.show_x + self.color_size * 1.2, self.show_y))
            self.show_x_list.append(self.show_x)
            self.show_x += self.rect_width
        #行动国家标记
        self.country_move()
        self.circle_x = self.show_x_list[self.countrymove-1] + 0.5 * self.color_size
        self.circle_y = self.show_y + self.color_size * 1.2
        #print(self.circle_x, self.circle_y)
        pg.draw.circle(screen, DEEPSKYBLUE, (self.circle_x, self.circle_y), 5)
        pg.display.flip()
        
#建立tile和land的对应关系
def TileLandDict(land, countland):
    for tile in land:
        tilelanddict[tile] = countland

landtiledict = {}
def LandTileDict(land, countland):
    landtiledict[countland] = land

def get_all_neighbours(x, y):
    neighbours = []
    if y % 2 == 0:  # 宽的一行
        steps = [(-1, 0), (-1, -1), (0, -1), (-1, 1), (1, 0), (0, 1)]
    else:  # 窄的一行
        steps = [(-1, 0), (1, 1), (0, -1), (1, -1), (1, 0), (0, 1)]

    for dx, dy in steps:
        nx, ny = x + dx, y + dy
        if 0 <= nx < width and 0 <= ny < height :
            neighbours.append((nx, ny))
    return neighbours

def get_neighbours(x, y):
    neighbours = []
    if y % 2 == 0:  # 宽的一行
        steps = [(-1, 1), (-1, 0), (-1, -1), (0, -1), (1, 0), (0, 1)]
    else:  # 窄的一行
        steps = [(0, 1), (-1, 0), (0, -1), (1, -1), (1, 0), (1, 1) ]

    for dx, dy in steps:
        nx, ny = x + dx, y + dy
        if 0 <= nx < width and 0 <= ny < height and bg_map[ny][nx] == 'EMPTY': #衡量边界条件还有已被选中条件
            neighbours.append((nx, ny))
    return neighbours


# 生成土地
def follow_generate_land(start_x, start_y, num_tiles): 
    land = {(start_x, start_y)}
    border_tiles = get_neighbours(start_x, start_y)
    
    #按照权重将tile添加到land中直到land大小达到num_tiles
    while len(land) < num_tiles:
        if not border_tiles:
            break  # 如果没有可用的相邻地块，则结束

        # 计算每个候选相邻地块的权重（它们与属于land的地块接壤的数量），依据权重生成地块
        weights = []
        for n in border_tiles:
            weight = sum((neighbour in land) for neighbour in get_neighbours(n[0], n[1]))
            weights.append(weight)

        # 根据权重随机选择一个相邻地块
        total_weight = sum(weights)
        if total_weight == 0:  # 防止除以0
            current_x, current_y = random.choice(border_tiles)
        else:
            weights = [w / total_weight for w in weights]  # 归一化权重
            current_x, current_y = random.choices(border_tiles, weights=weights, k=1)[0]
            #print("(current_x, current_y)",(current_x, current_y))

        # 添加到land集合中
        land.add((current_x, current_y))
        
        # 更新border_tiles
        border_tiles.remove((current_x, current_y))
        new_neighbours = get_neighbours(current_x, current_y)
        border_tiles.extend(n for n in new_neighbours if n not in land and n not in border_tiles)
        border_tiles = list(set(border_tiles))  # 移除重复项
        #print("border_tiles",border_tiles)
        #print()

    
    global countland #显示计数修改颜色来查看效果
    countland += 1
    #print(countland)
    landlist.append(land) #将新生成的land加入landlist中，之后在相邻土地没空间了开拓飞地有用
    TileLandDict(land, countland) #直接这样用函数靠谱吗？
    LandTileDict(land, countland) #这两步在定义相邻的地块时候有用，相当于未雨绸缪
    
    #生成土地实例
    landsdict[countland] = Land(countland, land, countryrandomlist[countland-1], dicerandomlist[countland-1])
    
    return land


def centerfinder(lastland): #依据上一块地来找新的中心点，具体思路如下
    #第一步：找到上一块土地所有neighbour的tile
    all_neighbours = set()
    for tile in lastland:
        for n in get_neighbours(tile[0], tile[1]):
            if bg_map[n[1]][n[0]] == 'EMPTY':  # 仅考虑空地
                all_neighbours.add(n)
    #第二步：随机选一个neighbour，检查与其连通的empty tile数量能不能到20以上；如果可以则选为center；
    #不行则换一个neighbour；所有neighbour都遍历完还不行，就换一块lastland
    for _ in range(len(all_neighbours)):
        potential_center = random.choice(list(all_neighbours))
        all_neighbours.remove(potential_center)
        # 检查从这个中心点开始能否找到足够的空地
        explored = {potential_center}
        border = set(get_neighbours(potential_center[0], potential_center[1]))
        
        while len(explored) < min_tiles and border:
            new_tile = border.pop()
            if bg_map[new_tile[1]][new_tile[0]] == 'EMPTY':
                explored.add(new_tile)
                border.update([n for n in get_neighbours(new_tile[0], new_tile[1]) if n not in explored])
    
            if len(explored) >= min_tiles:
                return potential_center  # 找到了足够大的区域
            
    
    return None

def nextcenterfinder(lastland):
    
    if centerfinder(lastland) != None:
        return centerfinder(lastland)
    elif centerfinder(lastland) == None:
        landlist.remove(lastland)
        if len(landlist) == 0:
            return None
        elif len(landlist) != 0:
            randomland = random.choice(landlist)
            nextcenter = nextcenterfinder(randomland)
           
            return nextcenter


def AllLandGenerator(starting_x, starting_y, clicklandnum):

    grid_x = starting_x
    grid_y = starting_y
    
    while countland < clicklandnum:
        num_tiles = random.randint(14, 16)
        landtiles = follow_generate_land(grid_x, grid_y, num_tiles)

        for tile_x, tile_y in landtiles:
            bg_map[tile_y][tile_x] = countryrandomlist[countland -1]
            
        #bg_map[grid_y][grid_x] = 'SELECT'

        newcenter = nextcenterfinder(landtiles)
        if newcenter != None:
            grid_x, grid_y = newcenter
            #bg_map[grid_y][grid_x] = 'SELECT'
        else:
            running = False
        
        drawBackgroundHex(screen, width, height, bg_map)
        pg.display.flip()

        pg.time.delay(10)  # 等待0.1秒

#通过这个函数来判定与一个land相互接壤的land
def LandAdj(landnum):
    innertilelist = [tile for tile in landtiledict[landnum]]
    adjlist = []
    for tile in innertilelist:
        tilebros = get_all_neighbours(tile[0], tile[1])
        for bro in tilebros:
            adjlist.append(tilelanddict[bro])
            
    adjset = set(adjlist)
    adjset.discard(0)
    adjset.discard(landnum)
    
    return adjset

def AllLandAdj(landnumber):
    landadjdict = {}
    for landnum in range(1, landnumber+1):
        landadj = LandAdj(landnum)
        landadjdict[landnum] = landadj
    return landadjdict


In [35]:
# 绘制六边形的黑色边框，一条边一条边地绘制
for y in range(height):
    for x in range(width):
        if y % 2 == 1 and x == width - 1:
            continue
        base_x, base_y = getHexMapPos(x, y)
        points = [
            (base_x, base_y + Y_LEN // 2 + Y_LEN),
            (base_x, base_y + Y_LEN // 2),
            (base_x + X_LEN, base_y),
            (base_x + X_LEN * 2, base_y + Y_LEN // 2),
            (base_x + X_LEN * 2, base_y + Y_LEN // 2 + Y_LEN),
            (base_x + X_LEN, base_y + Y_LEN * 2),
            (base_x, base_y + Y_LEN // 2 + Y_LEN)  # 再次添加第一个点，以便绘制最后一条边
        ]

        # 分别绘制每一条边
        for i in range(len(points) - 1):
            start_point = points[i]
            end_point = points[i + 1]
            pg.draw.line(surface, BLACK, start_point, end_point)


NameError: name 'Y_LEN' is not defined

In [14]:
###草稿
“”“
# 游戏主循环
countland = 0

running = True
while running:
    try:
        for event in pg.event.get():
            if event.type == pg.QUIT:
                running = False
            elif event.type == pg.MOUSEBUTTONDOWN:
                mouse_x, mouse_y = event.pos
                grid_x, grid_y = getHexMapIndex(mouse_x, mouse_y)
                num_tiles = random.randint(15, 16)
                landtiles = follow_generate_land(grid_x, grid_y, num_tiles)
                #print(landtiles)
                for tile_x, tile_y in landtiles:
                    if countland % 2 == 1:
                        bg_map[tile_y][tile_x] = 'ACTIVE'
                    else:
                        bg_map[tile_y][tile_x] = 'RANGE'
                
                bg_map[grid_y][grid_x] = 'SELECT'

                newcenter = centerfinder(landtiles)
                #print(newcenter)
                bg_map[newcenter[1]][newcenter[0]] = 'SELECT'
    except Exception as e:
        print(f"An error occurred: {e}", file=sys.stderr)
        running = False
            

    # 绘制六边形地图
    drawBackgroundHex(screen, width, height, bg_map)

    # 更新显示
    pg.display.flip()

# 退出 Pygame
pg.quit()
”“”

SyntaxError: invalid character '“' (U+201C) (1777797618.py, line 2)